Раздел: Начальные действия.
Пункт 4: Разработать и применить генератор тестовых данных.

In [3]:
import datetime
import random
import pyodbc
from faker import Faker

In [4]:



server = '.'  # Server name
database = 'PaymentData'  # Database name
user = 'SPUTNIKTECH\\zhout'  # User name
password = ''  # Password

# Create the connection string
cnxn_str = 'DRIVER={SQL Server};SERVER=' + server + ';DATABASE=' + database + ';Trusted_Connection=yes;UID=' + user + ';PWD=' + password

# Establish the connection
cnxn = pyodbc.connect(cnxn_str)
cursor = cnxn.cursor()

faker = Faker()

num_of_lines = 1


Заполняем PaymentParticipant:

In [5]:

import random
import datetime

def gener_date(start_time, end_time):
    timestamp = random.randint(int(start_time.timestamp()), int(end_time.timestamp()))
    return datetime.datetime.fromtimestamp(timestamp)


In [6]:

query = """
    INSERT INTO [dbo].[PaymentParticipant] ([Oid], [Balance], [Name], [OptimisticLockField], [GCRecord], 
    [ObjectType], [ActiveFrom], [InactiveFrom], [BankDetails], [Balance2], [Balance3]) 
    VALUES (NEWID(), ?, ?, ?, Null, ?, ?, ?, ?, ?, ?)
"""

# Prepare the parameter values as a list of tuples
values = []
for _ in range(num_of_lines):
    balance = 0
    balance2 = 0
    balance3 = 0
    name = faker.name()
    optimistic_lock_field = random.randint(1, 3)
    object_type = random.randint(0, 4)
    bank_details = faker.bban()
    active_from = gener_date(datetime.datetime(2010, 1, 1), datetime.datetime.now())
    inactive_from = gener_date(active_from, datetime.datetime.now())
    active_from = active_from.isoformat()
    inactive_from = inactive_from.isoformat()

    values.append((balance, name, optimistic_lock_field, object_type, active_from, inactive_from, bank_details, balance2, balance3))

# Execute the SQL statement with parameterized values
cursor.executemany(query, values)
cursor.commit()


In [7]:

# Prepare the SQL statement
query = "SELECT * FROM [dbo].[PaymentParticipant]"

# Execute the SQL statement and fetch all rows
cursor.execute(query)
rows = cursor.fetchall()

# Process the fetched rows
for row in rows:
    print(row)


('CF9AA63E-C3E2-49C2-B8B9-000AA31EA2DC', 0, 'Corey Figueroa', 2, None, 4, datetime.datetime(2012, 10, 28, 21, 19, 41), datetime.datetime(2015, 10, 22, 3, 4, 22), 'ZYQC28749260146980', 0, 0)
('75441DA5-72A4-48D9-A19E-0020C4D574BB', -99633, 'Aaron Cordova', 2, None, 4, datetime.datetime(2012, 1, 5, 7, 52, 26), datetime.datetime(2020, 7, 31, 3, 40, 24), 'WQDJ04476500754072', 0, 0)
('1AA11D8F-3F0D-4198-8FF5-002A5248D5D6', -92276, 'Margaret Howard', 3, None, 1, datetime.datetime(2014, 6, 13, 13, 21, 19), datetime.datetime(2015, 7, 25, 15, 0, 39), 'WASI76162401404603', 0, 0)
('395B2ADD-EA26-424B-8ED1-0032817FBAF7', 0, 'Savannah Roy', 2, None, 1, datetime.datetime(2014, 8, 13, 20, 30, 29), datetime.datetime(2015, 9, 4, 16, 55, 45), 'DQEJ75141478937506', 0, 0)
('B9C600FD-B21C-4D63-9CE9-003A9D4E5AD4', 0, 'Rebecca Phillips', 1, None, 3, datetime.datetime(2016, 6, 21, 14, 4, 50), datetime.datetime(2021, 2, 27, 14, 21, 1), 'WGSI68144500528536', 0, 0)
('BC1FE53C-978B-4046-9B5F-0047DED95DBD', 0, 'Ma

In [8]:

import uuid

num_of_lines = 1000

# Generate a list of tuples representing the data for each line
data = [
    (
        str(uuid.uuid4()),
        0,
        faker.name(),
        random.randint(1, 3),
        None,
        random.randint(0, 4),
        None,  # Placeholder for active_from
        None,  # Placeholder for inactive_from
        faker.bban(),
        0,
        0
    )
    for _ in range(num_of_lines)
]

# Assign values to active_from and inactive_from
for i, _ in enumerate(data):
    active_from = gener_date(datetime.datetime(2010, 1, 1), datetime.datetime.now())
    inactive_from = gener_date(active_from, datetime.datetime.now())
    data[i] = data[i][:6] + (active_from, inactive_from) + data[i][8:]

# Prepare the SQL statement
query = "INSERT INTO [dbo].[PaymentParticipant] ([Oid], [Balance], [Name], [OptimisticLockField], [GCRecord], " \
        "[ObjectType], [ActiveFrom], [InactiveFrom], [BankDetails], [Balance2], [Balance3]) " \
        "VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)"

# Execute the SQL statement for each line of data
cursor.executemany(query, data)

# Commit the transaction
cursor.commit()



Заполняем Supplier:
(PS: пусть object_type supplier'a = 0, object_type employer'a = 1, object_type client'a = 2, object_type cashbox'a = 3, object_type bank'a = 4)

In [9]:

supplier_obj_type = 0

# Retrieve participants with the specified ObjectType
query = f"SELECT Oid, Name FROM [dbo].[PaymentParticipant] WHERE ObjectType = {supplier_obj_type}"
cursor.execute(query)
participants = cursor.fetchall()

# Insert data into the Supplier table
for participant in participants:
    oid, contact = participant
    profit_by_material_as_payer = random.randint(0, 1)
    profit_by_material_as_payee = random.randint(0, 1)
    cost_by_material_as_payer = random.randint(0, 1)

    # Check if the record already exists in the Supplier table
    existing_query = f"SELECT COUNT(*) FROM [dbo].[Supplier] WHERE Oid = '{oid}'"
    cursor.execute(existing_query)
    count = cursor.fetchone()[0]

    if count == 0:
        insert_query = f"""INSERT INTO [dbo].[Supplier] ([Oid], [Contact], [ProfitByMaterialAsPayer], [ProfitByMaterialAsPayee], [CostByMaterialAsPayer])
                            VALUES (?, ?, ?, ?, ?)"""
        cursor.execute(insert_query, (oid, contact, profit_by_material_as_payer, profit_by_material_as_payee, cost_by_material_as_payer))

# Commit the transaction
cursor.commit()


In [10]:


query = "SELECT TOP (1) * FROM [dbo].[Supplier]"
cursor.execute(query)
row = cursor.fetchone()
while row:
    print(row)
    row = cursor.fetchone()


('BC1FE53C-978B-4046-9B5F-0047DED95DBD', 'Matthew Hughes', True, False, True)


Заполняем Employee:

In [11]:

import uuid

employee_obj_type = 1
query = f"SELECT Oid, Name FROM [dbo].[PaymentParticipant] WHERE ObjectType = {employee_obj_type}"
participants = cursor.execute(query)

for participant in participants.fetchall():
    oid, name = participant
    first_name, second_name = name.split(maxsplit=1)
    busy_until = gener_date(datetime.datetime(2010, 1, 1), datetime.datetime.now()).isoformat()
    stuff = random.randint(0, 100000)
    hour_price = random.randint(100, 2000)
    plan_fix_id = random.randint(1, 10000)
    plan_fix_money_request_task = faker.text(100)

    # Check if the employee already exists based on the Oid
    check_query = "SELECT COUNT(*) FROM [dbo].[Employee] WHERE [Oid] = ?"
    cursor.execute(check_query, (oid,))
    result = cursor.fetchone()

    if result[0] == 0:
        insert_query = """
            INSERT INTO [dbo].[Employee] ([Oid], [BusyUntil], [SecondName], [Stuff], [HourPrice], [Patronymic], 
            [PlanfixId], [Head], [PlanfixMoneyRequestTask])
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
        """
        cursor.execute(insert_query, (oid, busy_until, second_name, stuff, hour_price, first_name, plan_fix_id, None,
                                      plan_fix_money_request_task))
    else:
        print(f"Employee with Oid '{oid}' already exists.")

cursor.commit()





Employee with Oid '1AA11D8F-3F0D-4198-8FF5-002A5248D5D6' already exists.
Employee with Oid 'DDD1DE7F-6392-490A-9B60-0071A69E8924' already exists.
Employee with Oid '26FF851E-6C88-4E85-86AD-01A6BFDFFD34' already exists.
Employee with Oid '15033CD4-9988-4CCC-BFCE-01CF73A3082A' already exists.
Employee with Oid '1224F7DF-CF2B-4225-8759-02732AF132C7' already exists.
Employee with Oid '9867D50F-69D2-4B09-9460-033F0C5639E8' already exists.
Employee with Oid '2A833E88-9CD5-4B98-BC98-04B325CD59EB' already exists.
Employee with Oid '127EB5C7-3BFC-4789-94D7-04F1346FC0A0' already exists.
Employee with Oid '581E75EA-6B5D-4170-ACF0-0526405F1802' already exists.
Employee with Oid '7EEC3854-6087-46C7-B61A-05DD99C0DBBD' already exists.
Employee with Oid '559C4005-C722-4394-82D1-069C89791988' already exists.
Employee with Oid '8F54CEA2-49A0-458A-AC0B-07817D32F6ED' already exists.
Employee with Oid '32E58E1A-028A-4AB3-82EA-07E9C3534C17' already exists.
Employee with Oid '54735197-5B46-4FF1-B422-07F64497

In [12]:


query = "SELECT TOP (1) * FROM [dbo].[Employee]"
cursor.execute(query)

row = cursor.fetchone()
while row is not None:
    print(row)
    row = cursor.fetchone()


('1AA11D8F-3F0D-4198-8FF5-002A5248D5D6', datetime.datetime(2015, 8, 16, 10, 6, 43), 'Howard', 78102, 748, 'Margaret', 1311, None, 'Know really source main vote language together. Until national summer reflect collection grow sea.')


Заполняем Client:

In [13]:

client_obj_type = 2
query = f"SELECT Oid, Name FROM [dbo].[PaymentParticipant] WHERE ObjectType = {client_obj_type}"
cursor.execute(query)

participants = cursor.fetchall()
existing_oids = set()

for participant in participants:
    oid, name = participant
    first_name, second_name = name.split(maxsplit=1)
    phone_number = faker.phone_number()

    # Check if the OID already exists in the set of existing OIDs
    if oid in existing_oids:
        continue

    # Check if the record already exists in the Client table
    check_query = "SELECT COUNT(*) FROM [dbo].[Client] WHERE Oid = ?"
    cursor.execute(check_query, (oid,))
    count = cursor.fetchone()[0]

    if count > 0:
        # Record already exists, skip insertion
        existing_oids.add(oid)
        continue

    query = f"""INSERT INTO [dbo].[Client] ([Oid], [FirstName], [SecondName], [Phone])
                VALUES (?, ?, ?, ?)"""
    cursor.execute(query, (oid, first_name, second_name, phone_number))
    
    # Add the OID to the set of existing OIDs
    existing_oids.add(oid)

cursor.commit()



In [14]:


query = "SELECT TOP (1) * FROM [dbo].[Client]"
cursor.execute(query)

row = cursor.fetchone()
while row:
    print(row)
    row = cursor.fetchone()



('2E900495-3B9C-4E93-8ABF-0027158E7270', 'Lauren', 'Kennedy', '+1-892-592-2892')


Заполняем Project:

In [15]:

oid_employee = set()
query = "SELECT Oid FROM [dbo].[Employee]"
participants = cursor.execute(query)
for participant in participants:
    oid_employee.add(participant[0])

oid_client = set()
query = "SELECT Oid FROM [dbo].[Client]"
participants = cursor.execute(query)
for participant in participants:
    oid_client.add(participant[0])

existing_oids = set()
query_existing = "SELECT Oid FROM [dbo].[Project]"
existing_projects = cursor.execute(query_existing)
for project in existing_projects:
    existing_oids.add(project[0])

for _ in range(num_of_lines):
    address = faker.address()
    area = random.randint(1, 1000000)
    balance = 0
    balance_by_material = 0
    balance_by_work = 0
    client = random.choice(list(oid_client))
    start_date = gener_date(datetime.datetime(2010, 1, 1), datetime.datetime.now())
    finish_date = gener_date(start_date + datetime.timedelta(180), start_date + datetime.timedelta(3650))
    manager, foreman = random.sample(list(oid_employee), 2)
    name = faker.bs()
    optimistic_lock_field = random.randint(1, 3)
    plan_fix_change_request_task = faker.text(100)
    plan_fix_work_task = faker.text(100)
    remainder_the_advance = 0
    status = random.randint(0, 4)
    use_analytics = random.randint(0, 1)
    work_price_rate = random.uniform(100, 100000000)
    workers_price_rate = random.uniform(100, 500000)

    project_oid = str(uuid.uuid4())
    if project_oid in existing_oids:
        continue

    query = f"""INSERT INTO [dbo].[Project] ([Oid], [Name], [Address], [Client], [Manager], [Foreman], [OptimisticLockField], [GCRecord], 
                [Balance], [BalanceByMaterial], [BalanceByWork], [PlaningStartDate], [Status], [FinishDate], [Area], [WorkPriceRate],
                [WorkersPriceRate], [RemainderTheAdvance], [PlanfixWorkTask], [PlanfixChangeRequestTask], [UseAnalytics]) 
                VALUES (?, ?, ?, ?, ?, ?, ?, Null, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)"""

    cursor.execute(query, (project_oid, name, address, client, manager, foreman, optimistic_lock_field, balance,
                           balance_by_material, balance_by_work, start_date, status, finish_date, area, work_price_rate,
                           workers_price_rate, remainder_the_advance, plan_fix_work_task, plan_fix_change_request_task,
                           use_analytics))

    existing_oids.add(project_oid)

cursor.commit()



In [16]:

query = "SELECT TOP (1) * FROM [dbo].[Project]"
cursor.execute(query)

row = cursor.fetchone()
while row:
    print(row)
    row = cursor.fetchone()

('B4CA1887-A633-4C20-870B-0003824DE800', 'e-enable scalable e-services', '80519 Corey Canyon Suite 646\nLake Eric, AZ 81774', '9BEC2F79-EA62-4DEB-9ABB-7362F68B9B4F', '0E1132A9-A0EC-4CFA-A1FC-AD04645773BC', '79DA7188-B26E-4BCB-9B31-57EF7AB5F6B9', 2, None, 0, 0, 0, datetime.datetime(2012, 4, 4, 20, 56, 45), 4, datetime.datetime(2015, 2, 5, 15, 31, 22), 878812, 79978096.20214537, 322289.9289376183, 0, 'Talk bad natural father.', 'Office land long establish. Together enjoy the ago skin attention.', True)


Заполняем Cashbox:

In [17]:

cashbox_obj_type = 3
oid_acc_type = set()
query_acc_type = "SELECT Oid FROM [dbo].[AccountType]"
cursor.execute(query_acc_type)
participants_acc_type = cursor.fetchall()
for participant in participants_acc_type:
    oid_acc_type.add(participant[0])

existing_cashboxes = set()
query_existing = "SELECT Oid FROM [dbo].[Cashbox]"
cursor.execute(query_existing)
existing_cashbox_oids = cursor.fetchall()
for cashbox_oid in existing_cashbox_oids:
    existing_cashboxes.add(cashbox_oid[0])

query = f"SELECT Oid FROM [dbo].[PaymentParticipant] WHERE ObjectType = {cashbox_obj_type}"
cursor.execute(query)
participants = cursor.fetchall()
for participant in participants:
    if participant[0] in existing_cashboxes:
        continue

    account_type = random.choice(list(oid_acc_type))

    insert_query = "INSERT INTO [dbo].[Cashbox] ([Oid], [AccountType]) VALUES (?, ?)"
    cursor.execute(insert_query, (participant[0], account_type))

    existing_cashboxes.add(participant[0])

cursor.commit()



In [18]:

query = "SELECT TOP (1) * FROM [dbo].[Cashbox]"
cursor.execute(query)

row = cursor.fetchone()
while row:
    print(row)
    row = cursor.fetchone()

('B9C600FD-B21C-4D63-9CE9-003A9D4E5AD4', 'C14044C4-0D50-4BCE-AF32-0639F7738026')


Заполняем Bank:

In [19]:

bank_obj_type = 4
oid_acc_type = set()
query_acc_type = "SELECT Oid FROM [dbo].[AccountType]"
cursor.execute(query_acc_type)
participants_acc_type = cursor.fetchall()
for participant in participants_acc_type:
    oid_acc_type.add(participant[0])

existing_banks = set()
query_existing = "SELECT Oid FROM [dbo].[Bank]"
cursor.execute(query_existing)
existing_bank_oids = cursor.fetchall()
for bank_oid in existing_bank_oids:
    existing_banks.add(bank_oid[0])

query = f"SELECT Oid FROM [dbo].[PaymentParticipant] WHERE ObjectType = {bank_obj_type}"
cursor.execute(query)
participants = cursor.fetchall()
for participant in participants:
    if participant[0] in existing_banks:
        continue

    account_type = random.choice(list(oid_acc_type))

    insert_query = "INSERT INTO [dbo].[Bank] ([Oid], [AccountType]) VALUES (?, ?)"
    cursor.execute(insert_query, (participant[0], account_type))

    existing_banks.add(participant[0])

cursor.commit()


In [20]:


query = "SELECT TOP (1) * FROM [dbo].[Bank]"
cursor.execute(query)

row = cursor.fetchone()
while row:
    print(row)
    row = cursor.fetchone()

('CF9AA63E-C3E2-49C2-B8B9-000AA31EA2DC', 'C14044C4-0D50-4BCE-AF32-0639F7738026')


Заполняем Payment:

In [21]:

oid_category = set()
query_category = "SELECT Oid FROM [dbo].[PaymentCategory]"
cursor.execute(query_category)
participants_category = cursor.fetchall()
for participant in participants_category:
    oid_category.add(participant[0])

oid_participant = set()
query_participant = "SELECT Oid FROM [dbo].[PaymentParticipant]"
cursor.execute(query_participant)
participants_participant = cursor.fetchall()
for participant in participants_participant:
    oid_participant.add(participant[0])

oid_project = set()
query_project = "SELECT Oid FROM [dbo].[Project]"
cursor.execute(query_project)
participants_project = cursor.fetchall()
for participant in participants_project:
    oid_project.add(participant[0])

existing_payments = set()
query_existing = "SELECT Payer, Payee, Category, Project FROM [dbo].[Payment]"
cursor.execute(query_existing)
existing_payment_records = cursor.fetchall()
for record in existing_payment_records:
    existing_payments.add((record[0], record[1], record[2], record[3]))

for _ in range(num_of_lines):
    amount = random.randint(1, 100000)
    category = random.choice(list(oid_category))
    check_number = random.randint(10000, 99999)
    comment = faker.text(100)
    date = gener_date(datetime.datetime(2010, 1, 1), datetime.datetime.now()).isoformat()
    created_date = datetime.datetime.fromtimestamp(
        int(datetime.datetime.now().timestamp())
    ).isoformat()
    is_authorized = random.randint(0, 1)
    justification = faker.text(30)
    number = random.randint(10000, 99999)
    optimistic_lock_field = random.randint(1, 3)
    payee, payer = random.sample(list(oid_participant), 2)
    project = random.choice(list(oid_project))

    if (payer, payee, category, project) in existing_payments:
        continue

    insert_query = """
        INSERT INTO [dbo].[Payment] ([Oid], [Amount], [Category], [Project], [Justification], [Comment], [Date], [Payer], [Payee],
        [OptimisticLockField], [GCRecord], [CreateDate], [CheckNumber], [IsAuthorized], [Number])
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    """
    cursor.execute(insert_query, (str(uuid.uuid4()), amount, category, project, justification, comment, date, payer, payee,
                                  optimistic_lock_field, None, created_date, check_number, is_authorized, number))

    existing_payments.add((payer, payee, category, project))

cursor.commit()




In [22]:



query = "SELECT TOP (1) * FROM [dbo].[Payment]"
cursor.execute(query)

row = cursor.fetchone()
while row:
    print(row)
    row = cursor.fetchone()

('C65857E3-481F-4C40-A539-0013C5C713BD', 60600, '639D2087-D6B5-4FD0-AEFD-A572A4256CA3', 'B2E2B13B-385C-47E0-AD59-ED00A4C5B67A', 'Book sport question back.', 'Beyond record now table short nature give talk. Experience whose rich however.', datetime.datetime(2019, 10, 4, 23, 9, 59), '92B973CF-0DC3-4921-A1D9-7C732A07C0A7', 'BD08B6C8-4BB1-4B1C-AF7C-1C5D055C8911', 3, None, datetime.datetime(2023, 5, 31, 22, 10, 9), '24433', True, '36850')
